# Article, its date and its journal

In this jupyter notebook, we want to add the date and the journal for each article we have.
To do this, we ran two different scripts previously to obtain journal and date information:

- `article_information.py`
- `journals.py`

What we need now is to append this new information to our already created `article.csv` file.

In [1]:
import pandas as pd

In [2]:
art = pd.read_csv('/home/rdora/femec/data/article.csv')
art_supp = pd.read_csv('/home/rdora/femec/data/articles_supp.csv')
jnls = pd.read_csv('/home/rdora/femec/data/journals.csv')

/home/rdora/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We have tons of data, so we need to be careful as how we add it all together. To begin with, we need to deal with the repeated ids we have on some databases, particularly in the `art_supp` and the `art` data bases.

The goal here, is to have, for the maximum number of articles in `art`, its date, title and journal.

## First peak and unique articles

In [3]:
art_supp['handle'] = art_supp.handle.str.lower()

In [4]:
art_supp.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2193480 entries, 0 to 2193479
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   handle   2192682 non-null  object
 1   title    2180375 non-null  object
 2   year     2176892 non-null  object
 3   month    1391186 non-null  object
 4   jhandle  2192682 non-null  object
 5   jname    1536847 non-null  object
dtypes: object(6)
memory usage: 100.4+ MB


In [5]:
art_supp.nunique()

handle     2099360
title      1969762
year           965
month          857
jhandle       4330
jname         3754
dtype: int64

In [6]:
art.rename(columns={'Item-Id': 'handle'}, inplace=True)
uart = art[art['Item-Type']=='article'].handle.unique()

In [7]:
print(f"There are {len(uart):,} unique articles")

There are 489,091 unique articles


In [8]:
uart = pd.DataFrame({'handle': uart})

## 1. Removing missing data

In [9]:
art_supp_cp = art_supp.dropna(subset=['year', 'jname', 'handle'])

In [10]:
art_supp_cp.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1534104 entries, 1 to 2193478
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   handle   1534104 non-null  object
 1   title    1529557 non-null  object
 2   year     1534104 non-null  object
 3   month    1083685 non-null  object
 4   jhandle  1534104 non-null  object
 5   jname    1534104 non-null  object
dtypes: object(6)
memory usage: 81.9+ MB


In [11]:
art_supp_cp.nunique()

handle     1450517
title      1365641
year           787
month          773
jhandle       3838
jname         3740
dtype: int64

In [12]:
merge1 = pd.merge(uart, art_supp_cp, on='handle', how='left')

In [13]:
merge1.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500675 entries, 0 to 500674
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   handle   500675 non-null  object
 1   title    329811 non-null  object
 2   year     331160 non-null  object
 3   month    238936 non-null  object
 4   jhandle  331160 non-null  object
 5   jname    331160 non-null  object
dtypes: object(6)
memory usage: 26.7+ MB


In [14]:
merge1.nunique()

handle     489091
title      304309
year          213
month         533
jhandle      2732
jname        2824
dtype: int64

In [15]:
merge1.head()

,handle,title,year,month,jhandle,jname
0,repec:oup:jafrec:v:13:y:2004:i:02:p:ii1-ii14,Africa and the World Economy,2004,December,RePEc:oup:jafrec,Journal of African Economies
1,repec:eee:wdevel:v:31:y:2003:i:1:p:107-130,NaN,NaN,NaN,NaN,NaN
2,repec:eee:wdevel:v:28:y:2000:i:2:p:381-400,NaN,NaN,NaN,NaN,NaN
3,repec:eee:reveco:v:13:y:2004:i:3:p:325-340,NaN,NaN,NaN,NaN,NaN
4,repec:taf:jdevst:v:38:y:2001:i:2:p:27-56,Is Africa a Net Creditor? New Estimates of Cap...,2001,NaN,RePEc:taf:jdevst,Journal of Development Studies


In [16]:
merge1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500675 entries, 0 to 500674
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   handle   500675 non-null  object
 1   title    329811 non-null  object
 2   year     331160 non-null  object
 3   month    238936 non-null  object
 4   jhandle  331160 non-null  object
 5   jname    331160 non-null  object
dtypes: object(6)
memory usage: 26.7+ MB


In [17]:
art_supp_cp[art_supp_cp.handle=='repec:eee:reveco:v:13:y:2004:i:3:p:325-340']

,handle,title,year,month,jhandle,jname


## 2 Get journal and year from repec handle

In [18]:
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
def get_year(handle):
    reyear = re.compile(r'(?<=:y:)[0-9]+')
    s = handle.strip().split(':')
    if reyear.search(handle):
        year = int(reyear.search(handle).group())
        if 1900 > year or year > 2020:
            year = None
        elif datetime.date(1900, 1, 1) > datetime.date(year, 1, 1) > datetime.date(2020, 12, 1):
            year = None
    else:
        year = None
    
    return year

In [20]:
def get_journal(handle):
    reyear = re.compile(r'(?<=:y:)[0-9]+')
    s = handle.strip().split(':')
    if len(s) >= 3:
        handle_journal = ':'.join(s[:3])
    else:
        handle_journal = None
    
    return handle_journal

In [21]:
uart['year'] = uart['handle'].apply(get_year)
uart['journal'] = uart['handle'].apply(get_journal)

In [22]:
uart.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489091 entries, 0 to 489090
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   handle   489091 non-null  object 
 1   year     423246 non-null  float64
 2   journal  489091 non-null  object 
dtypes: float64(1), object(2)
memory usage: 11.2+ MB


In [23]:
uart.year.value_counts()

2013.0    21496
2014.0    21100
2012.0    20913
2015.0    20826
2016.0    20507
          ...  
1928.0        2
1929.0        1
1927.0        1
1926.0        1
1918.0        1
Name: year, Length: 96, dtype: int64

In [24]:
jnls.rename(columns={'handle': 'journal'}, inplace=True)

In [25]:
jnls.journal = jnls.journal.str.lower()

In [26]:
jnls = jnls[jnls.name.notna()]

In [27]:
jnls = jnls.drop_duplicates(subset=['journal'])

In [28]:
merge2 = pd.merge(uart, jnls, on='journal', how='left').drop('type', axis=1)

In [29]:
merge2.head()

,handle,year,journal,name
0,repec:oup:jafrec:v:13:y:2004:i:02:p:ii1-ii14,2004.0,repec:oup:jafrec,Journal of African Economies
1,repec:eee:wdevel:v:31:y:2003:i:1:p:107-130,2003.0,repec:eee:wdevel,World Development
2,repec:eee:wdevel:v:28:y:2000:i:2:p:381-400,2000.0,repec:eee:wdevel,World Development
3,repec:eee:reveco:v:13:y:2004:i:3:p:325-340,2004.0,repec:eee:reveco,International Review of Economics & Finance
4,repec:taf:jdevst:v:38:y:2001:i:2:p:27-56,2001.0,repec:taf:jdevst,Journal of Development Studies


## 3 Complete missing information obtained in 2

In [30]:
iyear = merge2[merge2.year.isna()].index
myears = merge2[merge2.year.isna()]['handle'].unique()

In [31]:
asup = art_supp_cp[art_supp_cp.handle.isin(myears)]

In [32]:
def remove_hyphen(year):
    y = year.split('-')
    
    return y[0]

In [33]:
asup.year = asup.year.astype(str)

/home/rdora/.local/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [34]:
asup.year = asup.year.apply(remove_hyphen)

asup.year = asup.year.replace({
    '2011.0': '2011',
    '2010.0': '2010',
    '2002/2003': '2002'})

asup = asup[asup.year != 'Forthcoming']

In [37]:
for year in asup.year.unique():
    try:
        int(year)
    except ValueError:
        print(year)

In [38]:
asup.year = asup.year.astype(int)

In [39]:
print(len(myears))

65845


In [40]:
asup = asup[(asup.year >= 1900) & (asup.year < 2021)]

In [41]:
asup = asup.drop_duplicates(subset=['handle'])

In [42]:
asup.nunique()

handle     40833
title      38620
year          68
month        207
jhandle       24
jname        671
dtype: int64

In [43]:
asup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40833 entries, 7992 to 2175371
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   handle   40833 non-null  object
 1   title    40831 non-null  object
 2   year     40833 non-null  int64 
 3   month    9569 non-null   object
 4   jhandle  40833 non-null  object
 5   jname    40833 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.2+ MB


In [44]:
asup = asup[['handle', 'year']]

In [45]:
merge3 = pd.merge(merge2[merge2.handle.isin(myears)].drop('year', axis=1),
                 asup, on='handle', how='left')

In [46]:
merge3 = pd.concat([merge2.drop(iyear), merge3])

In [47]:
merge3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489091 entries, 0 to 65844
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   handle   489091 non-null  object 
 1   year     464079 non-null  float64
 2   journal  489091 non-null  object 
 3   name     488243 non-null  object 
dtypes: float64(1), object(3)
memory usage: 18.7+ MB


In [48]:
merge3[merge3.name.isna()].journal.unique()

array(['repec:fce:ofcrev', 'repec:ieu:journl', 'repec:uwe:journl',
       'repec:sae:ecolab', 'repec:rau:homkmg', 'repec:srs:journl',
       'repec:ura:ecregj', 'repec:csb:stintr', 'repec:eyd:eyjrnl',
       'repec:ccp:journl', 'repec:mth:ifb888', 'repec:scn:0000mi',
       'repec:bpd:articl', 'repec:sae:emecst', 'repec:gig:jaaktu',
       'repec:kse:modern', 'repec:sae:crnind', 'repec:cos:jpaper',
       'repec:sae:ijcgvn', 'repec:gig:joupla', 'repec:mth:csbm88',
       'repec:mth:ijhr88', 'repec:wsd:irgpim', 'repec:pab:journl',
       'repec:iik:journl', 'repec:kse:chasop', 'repec:cnc:journl',
       'repec:nbp:journl', 'repec:env:journl', 'repec:fis:journl',
       'repec:epf:easopt', 'repec:rnp:artcls', 'repec:onl:scimod'],
      dtype=object)

In [49]:
jnls[jnls.journal == 'repec:mth:csbm88']

,journal,name,type


## Get missing information from `art_supp`

In [50]:
missing_journals = merge3[merge3.name.isna()].journal.unique()
missing_years = merge3[merge3.year.isna()].handle.unique()

In [51]:
art_supp.jhandle = art_supp.jhandle.str.lower()

In [52]:
art_supp = art_supp.rename(columns={'jhandle': 'journal',
                                    'jname': 'name'})

In [53]:
asup_j = art_supp[art_supp.journal.isin(missing_journals)][['journal', 'name']].dropna().drop_duplicates(subset='journal')

In [54]:
m3_j = pd.merge(merge3[merge3.name.isna()].drop('name', axis=1), asup_j,
               on='journal', how='left')

In [55]:
merge4 = pd.concat([merge3[merge3.name.notna()], m3_j])

In [56]:
merge4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489091 entries, 0 to 847
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   handle   489091 non-null  object 
 1   year     464079 non-null  float64
 2   journal  489091 non-null  object 
 3   name     488601 non-null  object 
dtypes: float64(1), object(3)
memory usage: 18.7+ MB


In [57]:
asup_y = art_supp[art_supp.handle.isin(missing_years)][['handle', 'year']].dropna().drop_duplicates(subset='handle')

In [72]:
asup_y.year = asup_y.year.astype(str)

In [74]:
asup_y.year = asup_y.year.apply(remove_hyphen)

asup_y.year = asup_y.year.replace({
    '2011.0': '2011',
    '2010.0': '2010',
    '2002/2003': '2002'})

asup_y = asup_y[asup_y.year != 'Forthcoming']
asup_y = asup_y[asup_y.year != '285']

In [75]:
for year in asup_y.year.unique():
    try:
        int(year)
    except ValueError:
        print(year)

In [76]:
m4_y = pd.merge(merge4[merge4.year.isna()].drop('year', axis=1), asup_y,
               on='handle', how='left')

In [77]:
merge5 = pd.concat([merge4[merge4.year.notna()], m4_y])

In [78]:
merge5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489091 entries, 0 to 25011
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   handle   489091 non-null  object
 1   year     487746 non-null  object
 2   journal  489091 non-null  object
 3   name     488601 non-null  object
dtypes: object(4)
memory usage: 18.7+ MB


## Add title to articles

In [79]:
art_supp.head()

,handle,title,year,month,journal,name
0,repec:bok:journl,NaN,NaN,NaN,repec,NaN
1,repec:bok:journl:v:24:y:2018:i:4:p:1-36,Monetary Policy and Corporate Investment,2018,December,repec:bok:journl,Economic Analysis (Quarterly)
2,repec:bok:journl:v:19:y:2013:i:4:p:47-86,Estimating the Neutral Real Interest Rate (NRI...,2013,December,repec:bok:journl,Economic Analysis (Quarterly)
3,repec:bok:journl:v:18:y:2012:i:1:p:1-29,Oil Price Shocks and the Macroeconomic Activity,2012,March,repec:bok:journl,Economic Analysis (Quarterly)
4,repec:bok:journl:v:21:y:2015:i:1:p:65-100,"Nominal Wage Contracts, Search Frictions and t...",2015,March,repec:bok:journl,Economic Analysis (Quarterly)


In [80]:
asup_t = art_supp[['handle', 'title']].dropna().drop_duplicates(subset='handle')

In [81]:
merge6 = pd.merge(merge5, asup_t, on='handle', how='left')

In [82]:
merge6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489091 entries, 0 to 489090
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   handle   489091 non-null  object
 1   year     487746 non-null  object
 2   journal  489091 non-null  object
 3   name     488601 non-null  object
 4   title    484413 non-null  object
dtypes: object(5)
memory usage: 22.4+ MB


In [96]:
merge6.to_csv('/home/rdora/femec/data/processed/unique_articles.csv', index=False)

In [95]:
merge6.head()

,handle,year,journal,name,title
0,repec:oup:jafrec:v:13:y:2004:i:02:p:ii1-ii14,2004,repec:oup:jafrec,Journal of African Economies,Africa and the World Economy
1,repec:eee:wdevel:v:31:y:2003:i:1:p:107-130,2003,repec:eee:wdevel,World Development,Public Debts and Private Assets
2,repec:eee:wdevel:v:28:y:2000:i:2:p:381-400,2000,repec:eee:wdevel,World Development,Financial Determinants of Domestic Investment ...
3,repec:eee:reveco:v:13:y:2004:i:3:p:325-340,2004,repec:eee:reveco,International Review of Economics & Finance,Additionality of debt relief and debt forgiven...
4,repec:taf:jdevst:v:38:y:2001:i:2:p:27-56,2001,repec:taf:jdevst,Journal of Development Studies,Is Africa a Net Creditor? New Estimates of Cap...


## Add information to article

In [89]:
art = art[art['Item-Type']=='article'].drop('Item-Type', axis=1)

In [91]:
merge7 = pd.merge(art, merge6, on='handle', how='left')

In [93]:
merge7.head()

,Author-Handle,Short-Id,handle,year,journal,name,title
0,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:oup:jafrec:v:13:y:2004:i:02:p:ii1-ii14,2004,repec:oup:jafrec,Journal of African Economies,Africa and the World Economy
1,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:eee:wdevel:v:31:y:2003:i:1:p:107-130,2003,repec:eee:wdevel,World Development,Public Debts and Private Assets
2,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:eee:wdevel:v:28:y:2000:i:2:p:381-400,2000,repec:eee:wdevel,World Development,Financial Determinants of Domestic Investment ...
3,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:eee:reveco:v:13:y:2004:i:3:p:325-340,2004,repec:eee:reveco,International Review of Economics & Finance,Additionality of debt relief and debt forgiven...
4,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:taf:jdevst:v:38:y:2001:i:2:p:27-56,2001,repec:taf:jdevst,Journal of Development Studies,Is Africa a Net Creditor? New Estimates of Cap...


In [92]:
merge7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 673580 entries, 0 to 673579
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Author-Handle  673580 non-null  object
 1   Short-Id       673580 non-null  object
 2   handle         673580 non-null  object
 3   year           671907 non-null  object
 4   journal        673580 non-null  object
 5   name           673007 non-null  object
 6   title          667755 non-null  object
dtypes: object(7)
memory usage: 41.1+ MB


In [97]:
merge7.to_csv('/home/rdora/femec/data/processed/article.csv', index=False)

In [98]:
merge7.head()

,Author-Handle,Short-Id,handle,year,journal,name,title
0,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:oup:jafrec:v:13:y:2004:i:02:p:ii1-ii14,2004,repec:oup:jafrec,Journal of African Economies,Africa and the World Economy
1,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:eee:wdevel:v:31:y:2003:i:1:p:107-130,2003,repec:eee:wdevel,World Development,Public Debts and Private Assets
2,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:eee:wdevel:v:28:y:2000:i:2:p:381-400,2000,repec:eee:wdevel,World Development,Financial Determinants of Domestic Investment ...
3,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:eee:reveco:v:13:y:2004:i:3:p:325-340,2004,repec:eee:reveco,International Review of Economics & Finance,Additionality of debt relief and debt forgiven...
4,repec:per:2004-04-21:leonce_ndikumana,pnd1,repec:taf:jdevst:v:38:y:2001:i:2:p:27-56,2001,repec:taf:jdevst,Journal of Development Studies,Is Africa a Net Creditor? New Estimates of Cap...
